In [ ]:
-- Using Database AdventureWorksDW2020, table dbo.FactInternetSales, write a query that will query Rention Cohort Analysis based on First time Customer Purchase in the period of Jan 2020 to Jan 2021

-- Tính chỉ số R, F, M
WITH CustSales AS(
SELECT
   CustomerKey,
   COUNT(DISTINCT SalesOrderNumber) AS Frequency,
   MAX(OrderDate) AS ClosestOrderDate,
   SUM(SalesAmount) AS Monetary,
   DATEDIFF(DAY, MAX(OrderDate), (SELECT MAX(OrderDate) FROM FactInternetSales)) AS Recency
FROM FactInternetSales
GROUP BY CustomerKey
),

-- Tính điểm từ 1 đến 4
rfm AS (
SELECT
    CustomerKey,
    NTILE(4) OVER (ORDER BY Recency DESC) AS R,
    NTILE(4) OVER (ORDER BY Frequency) AS F,
    NTILE(4) OVER (ORDER BY Monetary) AS M
FROM CustSales
),

rfm_score AS (SELECT
    CustomerKey,
    CONCAT(NTILE(4) OVER (ORDER BY Recency DESC), NTILE(4) OVER (ORDER BY Frequency), NTILE(4) OVER (ORDER BY Monetary)) AS rfm_score
FROM CustSales
),

-- Đặt điều kiện
CustSegmentation AS (SELECT 
    *,
    CASE 
        WHEN rfm_score LIKE '1__' THEN 'Lost Customer'
        WHEN rfm_score LIKE '[3,4][3,4][3,4]' THEN 'Loyal Customer'
        WHEN rfm_score LIKE '[3,4][3,4][1,2]' THEN 'Promising'
        WHEN rfm_score LIKE '_[1,2]4' THEN 'Big spender'
        WHEN rfm_score LIKE '[3,4][1,2]_' THEN 'New customer'
        WHEN rfm_score LIKE '2__' THEN 'Potiential churn'
    END AS CustomerSegmentation
FROM rfm_score)

SELECT
    COUNT(CustomerKey) AS NoofCustomer,
    CustomerSegmentation
FROM CustSegmentation
GROUP BY CustomerSegmentation